In [75]:
from utils.spark_utils import get_spatial_spark_session
from utils.params import CENS_STAGING_PATH
from pyspark.sql import functions as f

In [2]:
spark=get_spatial_spark_session("PostCodePolygon")

23/02/08 21:44:44 WARN Utils: Your hostname, DESKTOP-9QGN9QQ resolves to a loopback address: 127.0.1.1; using 172.17.116.103 instead (on interface eth0)
23/02/08 21:44:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/02/08 21:44:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
postcodePolyDf=spark.read.load("/mnt/c/Users/lohith/Downloads/OS_data_sets/processed/postcode_polygons")

In [78]:
crime_index_query = "select distinct lat, lon, lsoa_cd as lsoa11cd from crime_agg_raw "
_crime_index_pl = pl_df_from_query(crimeCon, crime_index_query)
crime_index_sdf = spark.createDataFrame(_crime_index_pl.to_dicts())

In [64]:
lsoa_list_34753 = list(_crime_index_pl.select(pl.col('lsoa11cd').unique()).to_series())

In [81]:
lsoa_list_34753[1]

'E01015709'

In [76]:
oa_spark_geo = spark.read.parquet(f"{CENS_STAGING_PATH}/geo_parquets/oa/").withColumn("poly_4326", f.expr("ST_FlipCoordinates(poly_4326_fc)"))

In [79]:
oa_spark_geo, crime_index_sdf

(DataFrame[OBJECTID: string, OA21CD: string, Shape__Are: string, Shape__Len: string, GlobalID: string, poly_4326_fc: udt, poly_4326: udt],
 DataFrame[lat: double, lon: double, lsoa11cd: string])

In [86]:
for LSOA_ITER in lsoa_list_34753[:2]:
    oa_spark_geo.alias('geo').join(
        f.broadcast(crime_index_sdf.filter(f"lsoa11cd='{LSOA_ITER}'")).alias('crimeIndex'),
        f.expr("""
            ST_Contains(geo.poly_4326, ST_Point(crimeIndex.lat, crimeIndex.lon)) and
        """),
        how="inner"
    )\
    .selectExpr(
        "OA21CD as oa21cd",
        "lat",
        "lon",
        "lsoa11cd"
    )\
    .write.mode("append")\
    .parquet(f"{CENS_STAGING_PATH}/geo_parquets/crime_pip_loop/")

23/02/08 22:43:17 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.
23/02/08 22:43:18 WARN TaskSetManager: Stage 23 contains a task of very large size (9806 KiB). The maximum recommended task size is 1000 KiB.


23/02/08 22:48:49 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


ERROR:root:KeyboardInterrupt while sending command.                 (0 + 0) / 4]
Traceback (most recent call last):
  File "/home/basal/envs/dbc1/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/basal/envs/dbc1/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/basal/miniconda3/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [14]:
import duckdb
import polars as pl
crimeCon = duckdb.connect("/home/basal/live_datasets/crimeapp.db")
censCon =  duckdb.connect("/home/basal/live_datasets/cens.db")
def pl_df_from_query(db_connection, query):
    return pl.from_arrow(db_connection.execute(query).arrow())

In [13]:
censCon.close()
crimeCon.close()

In [16]:
# I want lsoa11 to pc mapping

In [30]:
censCon.execute("drop table pcd_lsoa11cd_mapping").df()

,Success


In [35]:
censCon.execute("""
create or replace table pcd_to_lsoa11cd_mapping as 
with lsoa_map as (select distinct lsoa11_cd, lsoa21_cd from lsoa11_to_21)
select distinct replace(lower(pcd),' ','') as pcd, lsoa11_cd, lsoa21cd 
from 
pc_to_oa_mapping inner join 
lsoa_map on lsoa_map.lsoa21_cd = pc_to_oa_mapping.lsoa21cd
""")

In [43]:
censCon.execute("select cast(substring(lsoa11_cd, 2, length(lsoa11_cd)) as int)%1000  from pcd_to_lsoa11cd_mapping").df()

,"(CAST(main.substring(lsoa11_cd, 2, length(lsoa11_cd)) AS INTEGER) % 1000)"
0,616
1,616
2,616
3,743
4,732
...,...
2377341,620
2377342,621
2377343,623
2377344,639


In [19]:
censCon.execute("describe pc_to_oa_mapping").df()

,column_name,column_type,null,key,default,extra
0,pcd,VARCHAR,YES,NaN,NaN,NaN
1,gridgb1e,INTEGER,YES,NaN,NaN,NaN
2,gridgb1n,INTEGER,YES,NaN,NaN,NaN
3,oa21cd,VARCHAR,YES,NaN,NaN,NaN
4,lsoa21cd,VARCHAR,YES,NaN,NaN,NaN
5,lsoa21nm,VARCHAR,YES,NaN,NaN,NaN
6,msoa21cd,VARCHAR,YES,NaN,NaN,NaN
7,msoa21nm,VARCHAR,YES,NaN,NaN,NaN
8,ltla22cd,VARCHAR,YES,NaN,NaN,NaN
9,ltla22nm,VARCHAR,YES,NaN,NaN,NaN
